In [1]:
import os
import time
import sys
import json
import numpy as np
import ipyvolume as ipv
from pathlib import Path
import numpy as np
import random
import datetime

from tyssue import Sheet, config
from tyssue.io import hdf5
from tyssue.generation import ellipsoid_sheet
from tyssue.draw.ipv_draw import view_ipv
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.dynamics import effectors, units
from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics.factory import model_factory
from tyssue.solvers.sheet_vertex_solver import Solver
from tyssue.behaviors.events import EventManager
from tyssue.topology.sheet_topology import remove_face
from tyssue.utils.decorators import do_undo
from tyssue.utils import to_nd
from tyssue.io import hdf5


from invagination.ellipsoid import EllipsoidBGeometry as geom
from invagination.ellipsoid import VitellineElasticity
from invagination.delamination import (define_ovoid_mesoderm,
                                       initiate_monitoring_process,
                                       check_enter_in_process,
                                       check_tri_faces,
                                       type1_transition)

from invagination.basic_plot import mesoderm_position


import matplotlib.pyplot as plt

%matplotlib inline

SIM_DIR = Path('/home/guillaume/Simulations/mesoderm_invagination/')
SIM_DIR = Path('/home/admin-suz/Documents/Simulations')

today = datetime.date.today()

sim_save_dir = SIM_DIR/f'{today.isoformat()}_principal_results'
#sim_save_dir = SIM_DIR/f'2018-03-02'
try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

## Parameters

In [2]:
specs =  {
    'vert': {
        'z': 0.0,
        'x': 0.0,
        'is_active': 1,
        'y': 0.0,
        'rho': 0,
        'height': 0,
        'basal_shift': 0,
        'delta_rho': 30,
        'vitelline_K': 280.0,
        'radial_tension': 0},
    'face': {
        'z': 0.0,
        'x': 0.0,
        'num_sides': 6,
        'area': 1.0,
        'perimeter': 1.0,
        'is_alive': True,
        'y': 0.0,
        'contractility': 1.12,
        'prefered_area': 28,
        'area_elasticity': 1,
        'prefered_height': 32,
        'prefered_vol': 896
    },
    'cell': {
        'z': 0.0,
        'x': 0.0,
        'area': 0.0,
        'vol': 0.0,
        'num_faces': 6,
        'is_alive': True,
        'y': 0.0,
        'vol_elasticity': 5e-3,
        'prefered_vol': 4539601.384437251,
        'area_elasticity': 1.0,
        'prefered_area': 1.0
    },
    'edge': {
        'dz': 0.0,
        'ny': 0.0,
        'dx': 0.0,
        'nx': 0.0,
        'length': 0.0,
        'sub_vol': 0.0,
        'sub_area': 0.0,
        'srce': 0,
        'face': 0,
        'cell': 0,
        'dy': 0.0,
        'trgt': 0,
        'nz': 1.0,
        'line_tension': 0.0,
        'is_active': 1
    },
    'settings': {
        'abc': [8.5, 8.5, 15.0], # Ellipsoid axes
        'geometry': 'cylindrical',
        'height_axis': 'z',
        'nrj_norm_factor': 1,
        'vitelline_space': 0.2,
        'threshold_length': 1e-3,
        'critical_length': 1e-3
    }
}


## Epithelium generation

`tyssue` provides functions to create epithelium with various base geometries, such as planes or ellipses.

In [3]:
sheet = ellipsoid_sheet(*specs['settings']['abc'], 10)
print(f'The sheet has {sheet.Nf} vertices')

sheet.update_specs(specs)
ipv.clear()
ipv_fig, meshes = view_ipv(sheet, coords=list('zxy'),
                           color=sheet.vert_df.y)
ipv_fig

## Create a model


$$E = \sum_\alpha \frac{1}{2}\left(K_A(A - A_0)^2 + \Gamma L_\alpha^2\right) 
     + \sum_i \left(\delta_i R_i h_i + \frac{K_v}{2} r_i^2\right) + \frac{K_Y}{2}(V-V_0)^2$$ 

###### Define a new interaction


In [4]:
class RadialTension(effectors.AbstractEffector):
    
    dimensions = units.line_tension
    magnitude = 'radial_tension'
    label = 'Apical basal tension'
    element = 'vert'
    specs = {'vert':{'is_active',
             'height',
             'radial_tension'}}

    @staticmethod
    def energy(eptm):
        return eptm.vert_df.eval(
            'height * radial_tension * is_active')
         
    @staticmethod
    def gradient(eptm):
        grad = height_grad(eptm) * to_nd(
            eptm.vert_df['radial_tension'], 3)
        grad.columns = ['g'+c for c in eptm.coords]
        return grad, None
    


### Declarative model factory

In [5]:
model = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    #effectors.LineTension,
    effectors.FaceContractility,
    effectors.FaceAreaElasticity,
    effectors.CellVolumeElasticity,
    ], effectors.FaceAreaElasticity)


# Modify some initial value

sheet.cell_df['prefered_vol'] = sheet.cell_df['vol']
sheet.cell_df['vol_elasticity'] = 1e-3
sheet.face_df['prefered_area'] = sheet.face_df.area.mean()

geom.update_all(sheet)

### Gradient descent

In [6]:
solver_kw = {'minimize': {'method': 'L-BFGS-B',
                          'options': {'ftol': 1e-8,
                                      'gtol': 1e-8}}}

res = Solver.find_energy_min(sheet, geom, model, **solver_kw)
res.fun
print(res.message)
fig, ax = quick_edge_draw(sheet, coords=list('zx'))

In [7]:
ipv.clear()
ipv_fig, meshes = view_ipv(sheet, coords=list('zxy'),
                           color=sheet.vert_df.y)
ipv_fig

In [8]:

# Define rectangular mesoderm
#define_cell_in_mesoderm(sheet, 0.3, 0.7 ,0.05, 0.95)
    
# Define ovoid mesoderm
define_ovoid_mesoderm(sheet, 0, 0, 9, 4.)

mesoderm = sheet.face_df[sheet.face_df.is_mesoderm].index
delaminating_cells = sheet.face_df[sheet.face_df['is_mesoderm']].index



print('number of apoptotic cells: {}'.format(delaminating_cells.size))
mesoderm_position(sheet, delaminating_cells)

In [15]:

def run_sim(sheet, mesoderm, geom, model, dirname):
    
    
    delaminating_cells = []
    #Initiate manager
    manager = EventManager('face')
    sheet.face_df['enter_in_process'] = 0
    t=0
    stop = 40
    while manager.current and t < stop:
        # Clean radial tension on all vertices
        sheet.vert_df['radial_tension'] = 0
        manager.execute(sheet)
        res = Solver.find_energy_min(sheet, geom, model, **solver_kw)

        figname = os.path.join(
            dirname, 'invagination_{:04d}.png'.format(t))
        hdfname = figname[:-3]+'hf5'
        hdf5.save_datasets(hdfname, sheet)      
        
        # Add cells in delamination process if they are authorized
        check_enter_in_process(sheet, manager, mesoderm)

        # Add cells with initially 3 neighbourghs to be eliminated
        check_tri_faces(sheet, manager)
        # Add T1 transition for face with at least one edge shorter than critical length
        short_edges = sheet.edge_df['length'] < sheet.settings['critical_length']
        for face in sheet.edge_df[short_edges]['face'].unique():
            manager.append(type1_transition, face,
                           kwargs={'critical_length':0.3})    
    
        manager.update()
        
        ipv.clear()
        ipv_fig, meshes = view_ipv(sheet, coords=list('zxy'),
                                   color=sheet.vert_df.y)
        t+=1

    return sheet

In [16]:
def delamination_process(sheet, contractility_rate, critical_area, radial_tension):

    # Directory definition 
    dirname = '{}_contractility_{}_critical_area_{}_radialtension'.format(
                contractility_rate, critical_area, radial_tension)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass
    
    settings = {'contract_rate': contractility_rate,
                'critical_area': critical_area,
                'radial_tension': radial_tension,
                'nb_iteration':0,
                'contract_neighbors':False,
                'geom': geom}

    
    # Add some information to the sheet
    sheet2 = sheet.copy(deep_copy=True)
    
    sheet2.face_df['id'] = sheet2.face_df.index.values
    sheet2.settings['delamination'] = settings
    
    settings2 = {'critical_length':0.3}
    sheet2.settings['T1'] = settings2
    #""" Initiale find minimal energy
    # To be sure we are at the equilibrium
    res = Solver.find_energy_min(sheet2, geom, model, **solver_kw)
   
    sheet2 = run_sim(sheet2, delaminating_cells, geom, model, dirname)

    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')

In [17]:
delamination_process(sheet, 16, 1, 25)

In [21]:
dirname = '/home/guillaume/Simulations/mesoderm_invagination/2018-05-31_principal_results/16_contractility_1_critical_area_25_radialtension'
dirname = '/home/admin-suz/Documents/Simulations/2018-06-14_principal_results/16_contractility_1_critical_area_25_radialtension'

hfs = [f  for f in os.listdir(dirname) if f.endswith('hf5')]
hfs.sort()
def get_and_plot(dirname, hdf_file):
    dsets = hdf5.load_datasets(os.path.join(dirname, hdf_file),
                           data_names=['vert', 'edge', 'face'])

    specs = config.geometry.cylindrical_sheet()
    sheet = Sheet('ellipse', dsets, specs)

    fig, mesh = view_ipv(sheet, coords=list('zxy'),
                         color=sheet.vert_df.y*(sheet.vert_df.y>0))
    return fig, mesh

from ipywidgets import interact

def browse_sheets(dirname, hfs):
    n = len(hfs)
    ipv.clear()
    dsets = hdf5.load_datasets(os.path.join(dirname, hfs[0]),
                               data_names=['vert', 'edge', 'face'])
    specs = config.geometry.cylindrical_sheet()
    sheet = Sheet('ellipse', dsets, specs)
    fig, mesh = view_ipv(sheet, color=sheet.vert_df.y)
    fig.animation = 0
    def view_sheet(i):
        dsets = hdf5.load_datasets(os.path.join(dirname, hfs[i]),
                                   data_names=['vert', 'edge', 'face'])
        sheet = Sheet('ellipse', dsets, specs)
        
        mesh.lines = sheet.edge_df[['srce', 'trgt']].values
        mesh.x = sheet.vert_df.x.values
        mesh.y = sheet.vert_df.y.values
        mesh.z = sheet.vert_df.z.values
        spec = {'color': sheet.vert_df.y,
                'cmap': 'viridis'}
        mesh.color = 'g'#_color_from_sequence(spec, sheet)
        
    interact(view_sheet, i=(0, n-1))
    ipv.show()

In [22]:
browse_sheets(dirname, hfs)

### We're gonna need a bigger boat

In [27]:
dsets = hdf5.load_datasets('../data/hf5/ellipsoid_sheet_init.hf5',
                           data_names=['vert', 'edge', 'face', 'cell'])


with open('../data/json/ellipsoid.json', 'r+') as fp:
    big_specs = json.load(fp)

big_sheet = Sheet('ellipse', dsets, big_specs)

ipv.clear()
fig, mesh = view_ipv(big_sheet, color=big_sheet.vert_df.y)
fig